# Easy model:

$$\frac{\partial u^2}{\partial t^2} - \frac{\partial u^2}{\partial x^2} = f(t) \text{,} \qquad \forall x \in (0,  L) \text{,} \quad t \in (0, T)$$
$$u(0, t) = u(L, t) = 0 $$ 
$$u(x, 0) = u_{0}(x)$$
$$\frac{\partial u}{\partial t}(x, 0) = u_{1}(x)

# Weak form
let v be some test function: 
$$(v, u_{tt}) - (v, u_{xx}) = (v, f)$$
$$(v, u_{tt}) - vu_{x}\big |_{0}^{L} + (v_{x}, u_{x}) = (v, f)$$

find $ u \in V$ such that
$$(v, u_{tt}) + (v_{x}, u_{x}) = (v, f) $$

# Finite element approximation
Find $u_{h} \in V_{h}$ such that
$$(v, u_{h, tt}) + (v_{x}, u_{u, x}) = (v,f)$$
for all $ v \in V_{h} $

u and v expressed in terms of basis functions
$$u_{h}(x, t) = \sum_{j=1}^{N-1}\xi_{j}(t)\phi_{j}(x)$$
$$v(x, t) = \sum_{i=1}^{N-1}\alpha_{i}(t)\phi_{i}(x)$$ 

leading to

$$\sum_{j=1}^{N-1}(\phi_{i},\phi_{j})\xi^{''}(t) + \sum_{j=1}^{N-1}(\phi_{i}^{'},\phi_{j}^{'})\xi(t) = (\phi_{i}, f(t))$$ 

$$M\xi^{''}(t) + A\xi(t) = b(t)$$

In [2]:
import numpy as np
from scipy.sparse import dok_matrix
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

# Generateing Mass- & stiffnesmatrix and load vector with hat fucntions

In [3]:
def f(t):
    return t
 
#Create and return Mass, Stiffnes matrices and load vector
def matrix_conscution(x):
    N = len(x) - 1  # Number of elements
    A = np.zeros((N + 1, N + 1))  # Stiffness matrix
    M = np.zeros((N + 1, N + 1))  # Mass matrix
    B = np.zeros((N + 1, 1))  # Load vector
    
    # Element level stiffness and mass matrices for linear basis functions on a uniform mesh
    stepsize = x[1] - x[0]
    local_stiffness = np.array([[1, -1], [-1, 1]]) / stepsize
    local_mass = np.array([[2, 1], [1, 2]]) * (stepsize / 6)
    
    # Assemble global matrices
    for i in range(N):
        A[i:i+2, i:i+2] += local_stiffness
        M[i:i+2, i:i+2] += local_mass

    M[1,1] = M[N,N] = A[1, 1] = A[N,N] = 1e6
    return M, A

def load_vector(x, function):
    N = len(x) - 1  # Number of elements
    B = np.zeros((N + 1, 1))  # Load vector    
    stepsize = x[1] - x[0]

    for i in range(N):

        B[i] += function(x[i]) * stepsize / 2
        B[i+1] += function(x[i+1]) * stepsize / 2

    return B

# Generateing Mass- & stiffnesmatrix and load vector with hermite polynomials

In [89]:
#let xi be 
#xi = (x-x0)/(x1-x0)
a = 0
b = L = 1
N = 1000
h = (b-a)/N
x = np.linspace(a, b, N)
basis = {"eval": 
        {"H1": lambda x: 2*x**3 - 3*x**2, 
         "H2": lambda x: x**3 - 2*x**3 + x, 
         "H3": lambda x:  3*x**2 - 2*x**3, 
         "H4": lambda x: x**3 - x**2},
         "nabla": 
         {"H1_prime": lambda x: 6*x**2 - 6*x, 
          "H2_prime": lambda x: 3*x**2 - 6*x + 1,
          "H3_prime": lambda x: 6*x - 6*x**2,
          "H4_prime": lambda x: 3*x**2 - 2*x
         } 
}



M = np.zeros([N+1, N+1])
A = np.zeros([N+1, N+1])

for k in range(N-1):
        x0 = x[k]
        x1 = x[k+1]
        xi = (x-x0)/(x1-x0)
        quad = [(0, 1/6), (0.5, 4/6), (1, 1/6)]
        for base in basis.keys():   
            for i, H_i in enumerate(basis[base].values()):
                for j, H_j in enumerate(basis[base].values()):
                    for iter, weight in quad:
                        #idx_x, idx_y = k-i, k-j
                        #if idx_x and idx_y > 0 or idx_x and idx_y < N:
                            if base == "eval":
                                M[k - i, k - j] += h * weight * H_i(iter) * H_j(iter) 
                                
                            else:
                                A[k + i, k - j] += h * weight * H_i(iter) * H_j(iter)  
      

                

            
            


        


[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.70833333e-04 -4.16666667e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -4.16666667e-05  7.70833333e-04 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.33333333e-04
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


# Solve for first order ODE
$$ \xi^{'}(t) = w(t) $$
$$Mw^{'}(t) = b(t) - A\xi(t)
